# The TIF model iterator using pyspark

This notebook implements a tif iterator to use a sklearn model to predict every pixel in a raster .tif file.
This is done in a distributed way with pyspark because of the large number of pixels and time it will take without it.

It then outputs a segmentations shapefile which can be used in a GIS application to see the results.

Note MinMaxScalers have to be made in order to predict because of the differences between images.

Author: Michael de Winter

In [ ]:
import rasterio
import pandas as pd
import itertools
import numpy as np
import glob
import joblib
from pyspark.sql.functions import udf
import pickle
from shapely.geometry import Polygon, Point
from timeit import default_timer as timer
import geopandas as gpd
import os
from pyspark.sql.types import DoubleType, StringType, ArrayType
import glob
import shutil

###### run the below command to import all the required functions

In [ ]:
%run ./tif_model_iterator_functions

###Run settings

In [ ]:
# Coepelduynen
path_to_output = "/dbfs/mnt/PATH/TO/OUTPUT"
path_to_scalers = "/dbfs/mnt/PATH/TO/SCALERS"
parts = 2
model_path = "/dbfs/mnt/PATH/TO/MODEL" 

In [ ]:
# Predict all .tif files with a model.
for file  in glob.glob("/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/*.tif"):
  if file.split("/")[-1].split("_")[0] not in set(check_done_files()):
    try:
      print(file)
      run_tif_model_implementer(file,path_to_output,path_to_scalers,parts,model_path, aggregate_to_2m = False)
    except Exception as e:
      print(e)
  else:
      print(file.split("/")[-1].split("_")[0]+" already done")

20190302 already done
20190302 already done
20190308 already done
20190422 already done
20190601 already done
20191130 already done
20191202 already done
20200304 already done
20200323 already done
20200326 already done
20200508 already done
20200625 already done
20200731 already done
20200915 already done
20201231 already done
20210302 already done
20210423 already done
20210709 already done
20210815 already done
20210907 already done
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20211226_103526_SV1-01_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/databricks/python/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
 warnings.warn(
Total permutations this step: 15415981
Done with extracting dataframe in 8.513373872000102 second(s)
Filtering done in 38.381067487999644 second(s)
Filtered length of dataframe: 3535439
Normalization done in 38.55129070999965 second(s)
Finished making spark dataframe in 195.596199695 second(s)
Predicting finished in: 0.13339372999962507 second(s)
/databricks/spark/python/pyspark/sql/pandas/conversion.py:161: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.pyspark.fallback.enabled' does not have an effect on failures in the middle of computation.
 An error occurred while calling o1335.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:428)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:107)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:103)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: org.apache.spark.SparkException: Job aborted due to stage failure: Task 20 in stage 0.0 failed 4 times, most recent failure: Lost task 20.3 in stage 0.0 (TID 55) (10.0.4.6 executor 0): org.apache.spark.api.python.PythonException: 'NameError: name 'loaded_model' is not defined', from <command-206828270476792>, line 135. Full traceback below:
Traceback (most recent call last):
 File "<command-206828270476792>", line 135, in predict_pandas_udf
NameError: name 'loaded_model' is not defined

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:694)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:101)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:50)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:647)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknow